## Automate Event Planning

Maxim's remarks. 

- Prior to running the code... Fake agents pretending to be smth they are not, to do, what they are not equiped to do. I.e."Data Analyst" supposedly "uses statistical modeling and machine learning to provide crucial insights.". While this agent wan't be crunching any numbers, gathering datasets or fitting any models into the data I have no doubts the agent will generate some output which pretends to be the result of deep ML analysis of some data
- Thoughout the course I had this suspicion which I can now formulate. Agent descriptions and taks seemed redundant, i.e. the role description almost always is primed for a one single task that follows latter, not a single cases showed that an agent could be assignde multiple tasks. In the persepctive when every agent has one purpuse which is essentially it's task there's no point in a separate task
- 10 minutes and 315592 tokens latter (~$2) the GPT-4o agents have produced very shallow recoemndations which can be applied to any company, no need to run agents...
- It is clear that the study materials are not supposed to be production ready, complex and big but ratehr small and isolated to demonstrate the point... Yet it is also clear that the state of agentic development allows only narow application, we are not talking of any generic applicaitons to broader scope of tasks. What we see is a the ability to build a fragile chain of text generators, for some cases make a next level of automation possible

In [37]:
#!pip install -U crewai crewai_tools langchain_community

In [38]:
from crewai import Agent, Task, Crew
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

default_llm = AzureChatOpenAI(
    openai_api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-07-01-preview"),
    azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_key=os.environ.get("AZURE_OPENAI_KEY")
)

# from langchain_core.messages import HumanMessage
# message = HumanMessage(
#     content="Translate this sentence from English to French. I love programming."
# )
# r = default_llm.invoke([message])
# print(r)

## Tools

In [39]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Agents

In [40]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    llm = default_llm,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    llm = default_llm,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    llm = default_llm,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    llm = default_llm,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Tasks

In [41]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)

# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)

# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)


## Crew

- The Process class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [42]:
from crewai import Crew, Process

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
           
	# embedder={  
    #     "provider": "azure_openai",
    #     "config": {
    #         "model": "text-embedding-ada-002", 
    #         "deployment_name": "text-embedding-ada-002",
    #         "api_key": os.environ.get("AZURE_OPENAI_KEY")
    #     }
    # },
    
    manager_llm=default_llm,
    process=Process.hierarchical,
    verbose=True
)

2024-06-21 16:44:58,987 - 8693959680 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

In [43]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'EPAM',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (EPAM). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
Thought: To effectively monitor and analyze market data for EPAM, it's crucial to leverage the expertise of the team. We'll need the Data Analyst to gather and preprocess the market data, the Trading Strategy Developer to apply statistical modeling and machine learning techniques, the Trade Advisor to provide insights on the market trends, and the Risk Advisor to identify potential threats.

First, I'll delegate the task of gathering and preprocessing the market data to the Data Analyst.

Action: Delegate work to co-worker
Action Input: {"task": "Gather and preprocess market data for the selected stock (EPAM). This includes historical price data, trading volumes, financial statements, and any relevant news article

In [44]:
from IPython.display import Markdown
Markdown(result)

---

**Risk Analysis Report for EPAM Using the Reversal Trading Strategy**

**1. Introduction:**
This report provides a comprehensive risk analysis of the proposed trading strategies and execution plans for EPAM using the Reversal Trading strategy. It includes potential risks and suggested mitigation strategies, considering the current market conditions and optimal pricing.

**2. Detailed Execution Plan:**
The Reversal Trading strategy involves identifying points where the price of EPAM is expected to reverse direction. This strategy requires:
- Continuous monitoring of market conditions.
- Analysis of historical price patterns.
- Setting strict entry and exit criteria.
- Use of stop-loss orders to limit potential losses.

**3. Potential Risks:**
The following risks have been identified with the Reversal Trading strategy under the current market conditions:

**Volatile Market Conditions:**
- Unpredictable market movements can lead to significant losses.
- Price reversals might not occur as expected, causing trades to fail.

**Decreased Revenues and Operating Income:**
- Market conditions could adversely affect EPAM’s financial performance, impacting their ability to sustain operations.

**Foreign Exchange Risks:**
- Fluctuations in currency exchange rates can affect the profitability of trades.

**Operational Risks:**
- Risks related to internal processes, people, and systems.
- Potential disruptions in the trading infrastructure.

**Geopolitical Risks:**
- Political instability and changes in trade policies could impact market conditions.

**Regulatory Risks:**
- Changes in regulations could affect trading operations and compliance requirements.

**Technology and Cybersecurity Risks:**
- Risks of data breaches and cyber-attacks that could compromise trading operations.

**Reversal Trading Specific Risks:**
- False signals leading to incorrect trade entries and exits.
- Inadequate backtesting and simulation may result in untested strategies.

**4. Mitigation Strategies:**
To mitigate the identified risks, the following strategies are recommended:

**Volatile Market Conditions:**
- **Hedging Strategies:** Use options and futures to hedge against market volatility.
- **Diversification:** Spread investments across different sectors to mitigate risk.

**Decreased Revenues and Operating Income:**
- **Cost Management:** Implement strict cost control measures.
- **Revenue Diversification:** Explore new revenue streams to offset declines.

**Foreign Exchange Risks:**
- **Currency Hedging:** Use forward contracts and other derivatives.
- **Natural Hedging:** Align revenue and expenses in the same currency.

**Operational Risks:**
- **Risk Management Framework:** Establish a comprehensive framework for regular risk assessments.
- **Business Continuity Planning:** Maintain robust plans to ensure operations during disruptions.

**Geopolitical Risks:**
- **Geopolitical Analysis:** Monitor geopolitical developments and adjust strategies accordingly.
- **Geographic Diversification:** Diversify operations to reduce dependence on any single market.

**Regulatory Risks:**
- **Compliance Programs:** Ensure compliance with all relevant regulations.
- **Engage with Regulators:** Maintain open communication with regulatory bodies.

**Technology and Cybersecurity Risks:**
- **Advanced Cybersecurity Measures:** Implement robust cybersecurity measures.
- **Regular Audits:** Conduct regular security audits and vulnerability assessments.

**Reversal Trading Specific Risks:**
- **Strict Entry and Exit Criteria:** Define strict trade criteria to avoid false signals.
- **Stop-Loss Orders:** Use stop-loss orders to limit potential losses.
- **Backtesting and Simulation:** Conduct thorough backtesting using historical data.

**5. Conclusion:**
By implementing these mitigation strategies, EPAM can align its trading activities with its risk tolerance and navigate current market conditions more effectively. Regular reviews and updates to these recommendations are essential based on market changes and the company’s risk profile.

---

This comprehensive risk analysis report provides detailed insights into the potential risks associated with the Reversal Trading strategy for EPAM and offers actionable recommendations to mitigate these risks.

In [45]:
print(financial_trading_crew.usage_metrics)

{'total_tokens': 315592, 'prompt_tokens': 290937, 'completion_tokens': 24655, 'successful_requests': 61}
